# LEFT PRECONDITIONED GMRES

In [2]:
import numpy as np
from time import time
from numpy.linalg import qr

from functions import arnoldi_one_iter, back_substitution
from preconditioner import initial_precondition, PreconditionEnum

from scipy.sparse.linalg import lsqr
from scipy.sparse.linalg import spsolve
from scipy.sparse import csr_matrix
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import gmres
import scipy.io

In [3]:
def precon_GMRES_restarted(A, b, x0, k_max = None, restart = None, precondition = None, epsilon = 1e-12):
    """
    Generalized Minimal RESidual method for solving linear systems. With both restart and left preconditioning options.
    
    - Regular mode: when restart = None the full Krylov subspace is used to solve the problem. 
        This mode is guaranteed to find a solution (if one exists), but scales poorly.
        
    - Restarted mode: when restart = N then the Krylov subspace used by the solver is rebuilt every N steps. 
        This mode is not guaranteed to find a solution for certain intial conditions, but scales well.
    
    Parameters:
    -----------
    A : numpy.ndarray
        Coefficient matrix of the linear system.
        
    b : numpy.ndarray
        Right-hand side vector of the linear system.
        
    x0 : numpy.ndarray
        Initial guess for the solution.
        
    k_max : int
        Maximum number of iterations.
        
    restart : int, optional
        Number of iterations before restart. If None, the method will not restart.
        
    epsilon : float, optional
        Tolerance for convergence.
    
    Returns:
    --------
    numpy.ndarray
        Approximate solution to the linear system.
    """
    
    n = A.shape[0]
    
    if (k_max is None):
        k_max = n
        
    elif k_max > n:
        k_max = n
    
    r0 = b - A @ x0
    
    # Apply initial preconditioning
    M, r0, total_precondition_time = initial_precondition(A, precondition, r0) 
    
    p0 = np.linalg.norm(r0)
    beta = p0
    pk = p0
    ini= pk
    k = 0
    total_k = 0
    
    # Save list of errors at each iteration
    error_list = [pk]
    
    # Initialize the V basis of the Krylov subspace (concatenate as iteration continues). May terminate early.
    V = np.zeros((n, 1))
    V[:, 0] = r0 / beta
    
    # Hessenberg matrix
    H = np.zeros((n + 1, 1))
    
    while pk > epsilon*p0 and total_k < k_max:
        
        # Arnoldi iteration
        V = np.concatenate((V, np.zeros((n, 1))), axis=1)
        H = np.concatenate((H, np.zeros((n + 1, 1))), axis=1)
        H[:(k + 2), k], v_new, iter_precondition_time = arnoldi_one_iter(A, V, k, precondition = precondition, M = M)
        
        # update precondition time
        total_precondition_time += iter_precondition_time
        
        if v_new is None:
            print(f"ENCOUNTER EXACT SOLUTION")

            # Append 0 for plots...
            error_list.append(0)
            break
        else:
            V[:, k + 1] = v_new
        
        Q, R = qr(H[: k+ 2, :k +1], mode = 'complete')
        
        pk = abs(beta*Q[0, k]) # Compute norm of residual vector
        error_list.append(pk) # Add new error at current iteration       
        
        yk = back_substitution(R[:-1, :] , beta*Q[0][:-1])
    
        xk = x0 + V[:, :k + 1]@yk # Compute the new approximation x0 + V_{k}y

        k += 1
        total_k += 1
        
        if restart is not None and k == restart:
            x0 = xk
            r0 = b - A @ x0
            # Here is redundant to calculate the residual
            _, r0, iter_precondition_time = initial_precondition(A, precondition, r0) 
            total_precondition_time += iter_precondition_time
            
            p0 = np.linalg.norm(r0)
            beta = p0
            pk = p0
            k = 0
            
            V = np.zeros((n, 1))
            V[:, 0] = r0 / beta
            H = np.zeros((n + 1, 1))
  
    print(pk/ini)
    return xk, error_list, total_k, total_precondition_time

# Unit tests

## Test 1: Trying if the Jacobi preconditioner is formed correctly

In [34]:
A = np.array([[2,5,6],[2,3,4],[8,2,5]])

# Convert the matrix to CRS format
A = scipy.sparse.csr_matrix(A)

b = np.ones(A.shape[0])

x0 = np.zeros(b.size)

r0 = b - A @ x0

precondition = PreconditionEnum.JACOBI

M, r0, _ = initial_precondition(A, precondition, r0) # Skipping the time for now
M.toarray()

array([[2., 0., 0.],
       [0., 3., 0.],
       [0., 0., 5.]])

## Test 2: PDE sparse matrix comparison SciPy implementation

In [35]:
def discretise_poisson(N):
    """Generate the matrix and rhs associated with the discrete Poisson operator."""
    
    nelements = 5 * N**2 - 16 * N + 16
    
    row_ind = np.empty(nelements, dtype=np.float64)
    col_ind = np.empty(nelements, dtype=np.float64)
    data = np.empty(nelements, dtype=np.float64)
    
    f = np.empty(N * N, dtype=np.float64)
    
    count = 0
    for j in range(N):
        for i in range(N):
            if i == 0 or i == N - 1 or j == 0 or j == N - 1:
                row_ind[count] = col_ind[count] = j * N + i
                data[count] =  1
                f[j * N + i] = 0
                count += 1
                
            else:
                row_ind[count : count + 5] = j * N + i
                col_ind[count] = j * N + i
                col_ind[count + 1] = j * N + i + 1
                col_ind[count + 2] = j * N + i - 1
                col_ind[count + 3] = (j + 1) * N + i
                col_ind[count + 4] = (j - 1) * N + i
                                
                data[count] = 4 * (N - 1)**2
                data[count + 1 : count + 5] = - (N - 1)**2
                f[j * N + i] = 1
                
                count += 5
                                                
    return coo_matrix((data, (row_ind, col_ind)), shape=(N**2, N**2)).tocsr(), f

In [5]:
def jacobi_preconditioner(A):
    n = A.shape[0]
    diag = A.diagonal()
    M = csr_matrix((1/diag, (range(n), range(n))), shape=(n,n))
    return M


In [64]:
N = 200

A, b = discretise_poisson(N)

A = csr_matrix(A)

M = jacobi_preconditioner(A)

x0 = np.zeros(b.size)

r0 = b - A @ x0

maxiter = 1000
restart = 100
precondition = PreconditionEnum.JACOBI

In [65]:
start_time = time()
x, _, iterations, precontime = precon_GMRES_restarted(A, b, x0, k_max = maxiter, restart = restart, precondition = precondition, epsilon = 1e-18)
residual_calculated2 = np.linalg.norm(A@x - b)
end_time = time()
print("Optimized GMRES_restarted Time:", end_time - start_time)

print(f"Our implementation residual with Ax-b (max_iterations = {maxiter}, restart = {restart}): {residual_calculated2}")
print(iterations)
print(precontime)

4.0091664060633975e-05
Optimized GMRES_restarted Time: 27.32247805595398
Our implementation residual with Ax-b (max_iterations = 1000, restart = 100): 0.00793814948400553
1000
7.467154502868652


In [66]:
start_time = time()
x, _, iterations, precontime = precon_GMRES_restarted(A, b, x0, k_max = maxiter, restart = restart, precondition = None, epsilon = 1e-18)
residual_calculated2 = np.linalg.norm(A@x - b)
end_time = time()
print("Optimized GMRES_restarted Time:", end_time - start_time)

print(f"Our implementation residual with Ax-b (max_iterations = {maxiter}, restart = {restart}): {residual_calculated2}")
print(iterations)

4.009400588945499e-05
Optimized GMRES_restarted Time: 18.919415950775146
Our implementation residual with Ax-b (max_iterations = 1000, restart = 100): 0.007938613166112088
1000


In [ ]:
N = 200, maxiter = 300, no preconditioner time: 17.366547107696533, error: 0.0004307716389742024

In [8]:
N = 100

A, b = discretise_poisson(N)

A = csr_matrix(A)

M = jacobi_preconditioner(A)

x0 = np.zeros(b.size)

r0 = b - A @ x0

maxiter = 10000
restart = 10

start_time = time()
x, iterations = gmres(A, b, x0, M = None, restart = restart, maxiter = maxiter/restart, tol=1e-12)
residual_calculated1 = np.linalg.norm(A@x - b)
end_time = time()
print("SciPy GMRES Time:", end_time - start_time)

print(f"Calculated Scipy residual with Ax-b (max_iterations = {maxiter}, restart = {restart}): {residual_calculated1}")

start_time = time()
x = precon_GMRES_restarted(A, b, x0, k_max = maxiter, restart = restart, precondition = None)[0]
residual_calculated2 = np.linalg.norm(A@x - b)
end_time = time()
print("Optimized GMRES_restarted Time:", end_time - start_time)

print(f"Our implementation residual with Ax-b (max_iterations = {maxiter}, restart = {restart}): {residual_calculated2}")

SciPy GMRES Time: 1.5734102725982666
Calculated Scipy residual with Ax-b (max_iterations = 10000, restart = 10): 9.585398129745876e-11
Optimized GMRES_restarted Time: 7.398061275482178
Our implementation residual with Ax-b (max_iterations = 10000, restart = 10): 1.532266999365767e-11


## Test 2: Comparing SciPy implementation and our implementation in a matrix arised from a structural problem

In [6]:
# Load the .mtx file
A = scipy.io.mmread("data/bcsstk18.mtx")

M = jacobi_preconditioner(A)

# Convert the matrix to CSR format
A = csr_matrix(A)

b = np.ones(A.shape[0])

x0 = np.zeros(b.size)

maxiter = 200
restart = 50

start_time = time()
x, iterations = gmres(A, b, x0, M = None, restart = restart, maxiter = maxiter/restart)
residual_calculated1 = np.linalg.norm(A@x - b)
end_time = time()
print("SciPy GMRES Time:", end_time - start_time)

print(f"Calculated Scipy residual with Ax-b (max_iterations = {maxiter}, restart = {restart}): {residual_calculated1}")

start_time = time()
x = precon_GMRES_restarted(A, b, x0, k_max = maxiter, restart = restart, precondition = None)[0]
# x = custom_gmres(A, b, x0, num_max_iter= 200, precondition = PreconditionEnum.JACOBI)[0]
residual_calculated2 = np.linalg.norm(A@x - b)
end_time = time()
print("Optimized GMRES_restarted Time:", end_time - start_time)

print(f"Our implementation residual with Ax-b (max_iterations = {maxiter}, restart = {restart}): {residual_calculated2}")

SciPy GMRES Time: 0.2532637119293213
Calculated Scipy residual with Ax-b (max_iterations = 200, restart = 50): 83.92625488841975
0.7678037597305487
Optimized GMRES_restarted Time: 0.7207410335540771
Our implementation residual with Ax-b (max_iterations = 200, restart = 50): 83.92625444744131


### Now, let's try with the JACOBI preconditioner

In [14]:
start_time = time()
x, iterations = gmres(A, b, x0, M = M, restart = restart, maxiter = maxiter/restart)
residual_calculated1 = np.linalg.norm(A@x - b)
end_time = time()
print("SciPy GMRES Time:", end_time - start_time)

print(f"Calculated Scipy residual with Ax-b (max_iterations = {maxiter}, restart = {restart}, preconditioner = Jacobi): {residual_calculated1}")

start_time = time()
x = precon_GMRES_restarted(A, b, x0, k_max = maxiter, restart = restart, precondition = PreconditionEnum.JACOBI)[0]
residual_calculated2 = np.linalg.norm(A@x - b)
end_time = time()
print("Optimized GMRES_restarted Time:", end_time - start_time)

print(f"Our implementation residual with Ax-b (max_iterations = {maxiter}, restart = {restart}, preconditioner = Jacobi): {residual_calculated2}")

SciPy GMRES Time: 0.25827789306640625
Calculated Scipy residual with Ax-b (max_iterations = 200, restart = 50, preconditioner = Jacobi): 71.35722200521617
Optimized GMRES_restarted Time: 1.1413383483886719
Our implementation residual with Ax-b (max_iterations = 200, restart = 50, preconditioner = Jacobi): 71.35722100564513


The residual decreases! However, it takes longer, we need to optimize the code!

In [8]:
import scipy.io
import scipy.sparse

# Load the .mtx file
A = scipy.io.mmread("data/bcsstk18.mtx")

# Convert the matrix to CRS format
A = scipy.sparse.csr_matrix(A)

b = np.ones(A.shape[0])

x0 = np.zeros(b.size)

maxiter = 10000
restart = 20

start_time = time()
x, iterations = gmres(A, b, x0, restart = 20, maxiter = maxiter/restart)
residual_calculated1 = np.linalg.norm(A@x - b)
end_time = time()
print("SciPy GMRES Time:", end_time - start_time)

print(f"Calculated Scipy residual with Ax-b (max_iterations = {maxiter}, restart = {restart}): {residual_calculated1}")

SciPy GMRES Time: 5.927543640136719
Calculated Scipy residual with Ax-b (max_iterations = 10000, restart = 20): 70.57320010911614


In [9]:
start_time = time()
x = precon_GMRES_restarted(A, b, x0, k_max = 10000, restart = 20)[0]
residual_calculated2 = np.linalg.norm(A@x - b)
end_time = time()
print("Optimized GMRES_restarted Time:", end_time - start_time)

print(f"Our implementation residual with Ax-b (max_iterations = {maxiter}, restart = {restart}): {residual_calculated2}")

0.6456426396399392
Optimized GMRES_restarted Time: 16.155805587768555
Our implementation residual with Ax-b (max_iterations = 10000, restart = 20): 70.57320020880756


BEFORE IT TOOK 73 SECONDS. NOW 16!

In [10]:
start_time = time()
x = precon_GMRES_restarted(A, b, x0, k_max = 10000, restart = 20, precondition = PreconditionEnum.JACOBI)[0]
residual_calculated2 = np.linalg.norm(A@x - b)
end_time = time()
print("Optimized GMRES_restarted Time:", end_time - start_time)

print(f"Our implementation residual with Ax-b (max_iterations = {maxiter}, restart = {restart}): {residual_calculated2}")

9.163680759141599e-10
Optimized GMRES_restarted Time: 38.73997926712036
Our implementation residual with Ax-b (max_iterations = 10000, restart = 20): 0.30657085238777154


TOOK WAY LONGER WITH THE PRECONDITIONER BUT THE ERROR DECREASED LIKE CRAZY